In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights
from PIL import Image
import numpy as np
import requests
from io import BytesIO


In [2]:

# Define the FaceNet model with pre-trained ResNet18
class FaceNet(nn.Module):
    def __init__(self):
        super(FaceNet, self).__init__()
        # Load the pre-trained ResNet18 model with updated 'weights' argument
        self.resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # Remove the last fully connected layer (fc layer)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        # Get the number of input features for the new embedding layer from the original resnet.fc layer
        num_ftrs = models.resnet18(weights=ResNet18_Weights.DEFAULT).fc.in_features
        # Add a new fully connected layer for embeddings
        self.embedding_layer = nn.Linear(num_ftrs, 128)  # Output size of 128

    def forward(self, x):
        x = self.resnet(x)  # Get features from ResNet
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.embedding_layer(x)  # Get embeddings
        return x
    
    def get_face_embeddings(self, image_tensor):
        self.eval()  # Set model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation for inference
            embeddings = self(image_tensor)
        return embeddings

    

In [3]:


# Function to calculate cosine similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    return F.cosine_similarity(embedding1, embedding2)



In [4]:

# Preprocessing function to prepare images for the model
def preprocess_image_from_url(url):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing to ResNet's expected size
        transforms.ToTensor(),  # Convert image to Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for ResNet
    ])
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers, verify=False)
    
    # Check if the response is successful
    if response.status_code != 200:
        raise ValueError(f"Failed to fetch image from {url} (status code: {response.status_code})")
    
    # Check the content type
    content_type = response.headers.get('Content-Type')
    if 'image' not in content_type:
        raise ValueError(f"Expected image, but got {content_type} from {url}")
    
    # Open and preprocess the image
    img = Image.open(BytesIO(response.content)).convert('RGB')  # Open and convert to RGB
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    return img_tensor




In [ ]:
# Image URLs
url1 = "https://upload.wikimedia.org/wikipedia/commons/9/90/Brad_Pitt-69858.jpg"  # Brad Pitt
url2 = "https://upload.wikimedia.org/wikipedia/commons/7/71/George_Clooney-69990.jpg"  # George Clooney
# Get embeddings for both images
image_tensor1 = preprocess_image_from_url(url1)
image_tensor2 = preprocess_image_from_url(url2)

In [ ]:



# Load the FaceNet model
facenet_model = FaceNet()
facenet_model.eval()
embeddings1 = facenet_model.get_face_embeddings( image_tensor1)
embeddings2 = facenet_model.get_face_embeddings( image_tensor2)
# Compute the cosine similarity between the embeddings
similarity = cosine_similarity(embeddings1, embeddings2)
print(f"Cosine Similarity between the faces: {similarity.item()}")

In [ ]:
print(facenet_model)

In [ ]:
import torch
from torchinfo import summary  # For detailed model summary
from fvcore.nn import FlopCountAnalysis  # For FLOPS calculation

# Print model summary
summary(facenet_model, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params"], verbose=2)

# Calculate FLOPs using fvcore
input_tensor = torch.randn(1, 3, 224, 224)  # Example input tensor
flop_counter = FlopCountAnalysis(facenet_model, input_tensor)

# Show only inference FLOPs
total_flops = flop_counter.total()  # Get total FLOPs for inference
print(f"Total GFLOPs: {total_flops/ 1e9} GFLOPs")


In [ ]:

import torch
import torch.nn as nn
from torchvision import models
from torch.nn import functional as F
from torchprofile import profile_macs  # Assumes torchprofile is installed
from torchvision.models import ResNet18_Weights


# Step 2: Calculate FLOPs
# Example input tensor of the shape that ResNet18 expects (batch size = 1, 3 color channels, 224x224 image)
input_tensor = torch.randn(1, 3, 224, 224)

# Profile the model to calculate MACs/FLOPs
flops = profile_macs(facenet_model, input_tensor) * 2  # Convert MACs to FLOPs

# Convert FLOPs to GFLOPs
gflops = flops / 1e9
print(f"Total GFLOPs for inference: {gflops:.2f} GFLOPs")

In [ ]:
import torch
from torchinfo import summary  # For detailed model summary
import torchprofile as tp      # Import the module correctly

# Print model summary using torchinfo
summary(facenet_model, input_size=(1, 3, 224, 224))

# Use torchprofile to calculate FLOPs and parameters
#flops_ = tp.profile(facenet_model, input_size=(1, 3, 224, 224))  # Use the correct reference to the profile function
#print(f"Total FLOPs: {flops_}")





In [11]:
import torch
import torch.onnx

def export_model_to_onnx(model, input_size=(1, 3, 224, 224), output_path="facenet_resnet18.onnx"):
    """
    Export the PyTorch model to ONNX format without dynamic batching.
    
    Parameters:
    - model: The PyTorch model to export
    - input_size: The size of the input tensor (default is for ResNet18 input)
    - output_path: Path where the ONNX model will be saved
    """
    # Set the model to evaluation mode
    model.eval()
    
    # Create a dummy input tensor with the specified size
    dummy_input = torch.randn(input_size)
    
    # Export the model to ONNX
    torch.onnx.export(
        model,                               # Model to export
        dummy_input,                         # Dummy input tensor
        output_path,                         # Output file path
        export_params=True,                  # Store the trained parameter weights
        opset_version=11,                    # ONNX version to export to
        do_constant_folding=True,            # Whether to apply constant folding optimization
        input_names=['input'],               # Model's input name
        output_names=['output']              # Model's output name
    )
    
    print(f"Model exported to ONNX format and saved as {output_path}")




In [ ]:
import onnx
import pathlib
import os

# Define model path and ensure directory exists
model_path = "onnx/facenet_resnet18.onnx"
model_dir = os.path.dirname(model_path)

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

def export_model_to_onnx(model, input_size=(1, 3, 224, 224), output_path=model_path):
    dummy_input = torch.randn(*input_size)  # Dummy input for model tracing
    torch.onnx.export(model, dummy_input, output_path, opset_version=11)
    print(f"Model exported to {output_path}")

# Assuming facenet_model is defined and loaded with weights
export_model_to_onnx(facenet_model, input_size=(1, 3, 224, 224), output_path=model_path)

# Load and check the exported ONNX model
onnx_model = onnx.load(pathlib.Path(model_path))
onnx.checker.check_model(onnx_model)
print("ONNX model is valid.")


In [13]:
import netron
def show_model(model_file_name,itf='localhost',port=8098):
    netron.start(file=model_file_name,address=(itf,port))
    return port

In [ ]:
port=show_model("./"+str(model_path))